<a href="https://colab.research.google.com/github/toinnn/Chat_Bot/blob/master/Chat_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Enviroment Preparation :*

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()
!pip install pyspark py4j
%pip install jaro-winkler
# !cat /proc/cpuinfo
# !lscpu | grep 'Core(s) each processor has/per socket:'
# !lscpu | grep 'Number of threads/core:'

     |████████████████████████████████| 281.4 MB 41 kB/s 
     |████████████████████████████████| 198 kB 66.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=f7da2b8d6fa7bad452774adb85ebe51ae8a1a77b43b30e43df0f5aa8430c96ad
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [4]:
Git_Dir    = "/content/drive/MyDrive/Github_Dir/Chat_Bot"
Neural_Dir = "Machine-Learning__Deep-Learning"
with open("/content/drive/MyDrive/Github_Dir/acess_Token_Git.txt","r") as file:
    acess_Token_Git = file.read()
Git_Path   = "https://"+ acess_Token_Git + "@github.com/toinnn/" + Neural_Dir + ".git"
Git_CB_Path= "https://"+ acess_Token_Git + "@github.com/toinnn/" + "Chat_Bot" + ".git"

In [4]:
!git clone "{Git_Path}" ./temp/Machine-Learning__Deep-Learning
!git clone "{Git_CB_Path}" ./temp/Chat_Bot

!mv ./temp/* "{Git_Dir}"
!rm -rf ./temp 

!rsync -aP "{Git_Dir}"/*  ./
#!ln -s "/content/drive/MyDrive/Github_Dir/Chat_Bot" + Neural_Dir NLP

Cloning into './temp/Machine-Learning__Deep-Learning'...
remote: Enumerating objects: 530, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 530 (delta 12), reused 10 (delta 1), pack-reused 496
Receiving objects: 100% (530/530), 49.56 MiB | 11.01 MiB/s, done.
Resolving deltas: 100% (247/247), done.
Cloning into './temp/Chat_Bot'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 118 (delta 52), reused 20 (delta 6), pack-reused 0
Receiving objects: 100% (118/118), 1.57 MiB | 5.30 MiB/s, done.
Resolving deltas: 100% (52/52), done.
mv: inter-device move failed: './temp/Chat_Bot' to '/content/drive/MyDrive/Github_Dir/Chat_Bot/Chat_Bot'; unable to remove target: Directory not empty
mv: inter-device move failed: './temp/Machine-Learning__Deep-Learning' to '/content/drive/MyDrive/Github_Dir/Chat_Bot/Machine-Learning__Deep-Lea

### \> *Data* *Preparation :*

In [5]:
from joblib.parallel import Parallel
%cd "/content/{Neural_Dir}"
from BiLSTM import BiLSTM ,BiLSTM_Attention
%cd /content/Chat_Bot
from pyspark.sql import SparkSession
from pyspark.sql.functions import col , row_number
from pyspark.sql.window import Window
import os
import csv
import pickle
import joblib
from joblib import Parallel, delayed
import re


/content/Machine-Learning__Deep-Learning
teste
/content/Chat_Bot


* *Load & Clean Dataset :*

In [ ]:


# spark = SparkSession.builder.getOrCreate()
# df = spark.read.csv("/content/Chat_Bot/Kaggle_Twitter_Custumer_Support__archive/twcs/twcs.csv" , header = True )

# df = df.filter( (df.tweet_id >= 1) & (df.text != "") )

# df.write.format('csv').option('header',
#                                True).mode('overwrite').option('sep',
#                                                               ',').save("/content/drive/MyDrive/Github_Dir/Chat_Bot/Chat_Bot/teste")
# os.system("cat /content/drive/MyDrive/Github_Dir/Chat_Bot/Chat_Bot/teste/p* > /content/drive/MyDrive/Github_Dir/Chat_Bot/Chat_Bot/teste.csv")
# df.coalesce(1).write.format('csv').option('header',True).mode("overwrite").save('/content/drive/MyDrive/Github_Dir/Chat_Bot/Chat_Bot/result3.csv')
# df.show(40)




In [ ]:
# with open('/content/drive/MyDrive/Github_Dir/Chat_Bot/Chat_Bot/result3.csv/part-00000-94f3bc51-847d-44a3-a9d9-87a09b5284e8-c000.csv', 'r') as file:
#     cs = csv.reader(file)
#     cs = { i[0]:i for i in cs}

# pickle.dump(cs , open("/content/drive/MyDrive/Github_Dir/Chat_Bot/Chat_Bot/datasetDict.pickle","wb"))
path = "/content/drive/MyDrive/Github_Dir/Chat_Bot/Chat_Bot/datasetDict.pickle"
# cs = pickle.load(open(path,"rb"))

def loadPickle(path):
    with open(path , "rb") as pt :
        return joblib.load(pt)
data_dict = loadPickle(path)
# cs = loadPickle(path)


print(data_dict["tweet_id"])
print(data_dict["1"])


['tweet_id', 'author_id', 'inbound', 'created_at', 'text', 'response_tweet_id', 'in_response_to_tweet_id']
['1', 'sprintcare', 'False', 'Tue Oct 31 22:10:47 +0000 2017', '@115712 I understand. I would like to assist you. We would need to get you into a private secured link to further assist.', '2', '3']


In [ ]:
cvv = {}
for i in list(data_dict.items()):
    try :
        cvv[i[0]] = (i[1][1] , i[1][4] , i[1][5] , i[1][6] )
    except :
        print(i)

print(cvv["tweet_id"])
cs = None
data_dict = None
# pickle.dump(cvv , open("/content/drive/MyDrive/Github_Dir/Chat_Bot/Chat_Bot/datasetDict_Sucinto.pickle","wb"))


('172981', ['172981', '156475', 'True', 'Fri Nov 24 22:16:01 +0000 2017', '@AppleSupport hey! I get a message on #macOS when I connect my #iPhoneX to #MacBookPro (15\\",2017) using USB-C to #Lightning cable - it says it draws too much power??!! Should I be concerned??\\""', '172979'])
('486515', ['486515', 'marksandspencer', 'False', 'Fri Dec 01 15:43:09 +0000 2017', '@230767 Hi Patricia, we offer 29\\",31\\"\\" and 33\\"\\" lg lengths"', 'along with 27\\\\" in our petite range. We used to offer XL but these didn\'t sell as well as we\'d hoped. We\'ll tell our buyers you\'d like to see more longer"'])
('637668', ['637668', '271197', 'True', 'Wed Nov 22 20:59:28 +0000 2017', '@SpotifyCares I have a 13\\",mid-2012 Macbook pro. if you need processor/RAM specs I can also provide that.\\""', '637670'])
('1054460', ['1054460', '369154', 'True', 'Sat Oct 14 20:51:16 +0000 2017', '@115900 Really? Why we can\'t watch the game this afternoon? You keep having “ problems with the program \\",can\'

In [ ]:
def clean_Text( data : dict , minSize = .6 ) -> dict :
    for_delete = []
    for i in data.keys() :
        # decode_word = []
        text = data[i][1].split()
        initial_Size = len(text)
        phrase = ""
        counter = 0
        for word in text :
            try :
                # decode_string.update({i.encode('latin-1', "ignore").decode(): None}) 
                var = word.encode('latin-1', "ignore").decode()
                if var != "" :
                    phrase += " " + var
                    counter += 1
            except :
                pass
        if counter/initial_Size > minSize :
            data.update({i : (data[i][0] , phrase , data[i][2] , data[i][3] )})
        else :
            for_delete += [i]
    for i in for_delete :
        del data[i]  
    return data      


cvv = pickle.load(open("/content/drive/MyDrive/Github_Dir/Chat_Bot/Chat_Bot/datasetDict_Sucinto.pickle","rb"))
print("cvv antes : ", len(cvv))
cvv = clean_Text(cvv )
print("cvv depois : ", len(cvv))
#Regex : /^\d+(,\d+)*$/
def str2list(input : str )-> list :
    return re.findall( r'\d+' , input )
def up_catch_talk(data : dict , id : int , output:dict = {} , down : bool = True) -> dict :
    try :
        var = data[str(id)]        
        try :
            var = output[id]
            return
        except :
            try:
                for i in str2list(data[str(id)][2]) :
                    up_catch_talk(data , int(i) , output , up = False )
                    output.update({id : (data[str(id)][1] , [output[int(i)][0]] + output[int(i)][1]  ) })
            except:
                output.update({id : ( data[str(id)][1] , [] ) })
        if down == True :
            for i in str2list(data[str(id)][3]) :
                up_catch_talk(data , int(i) , output , down = False )
    except :
        return
def down_catch_talk(data : dict , id : int , output:dict = {} , up : bool = True) -> dict :
    try :
        var = data[str(id)]
        try :
            var = output[id]
            return
        except :
            try:
                for i in str2list(data[str(id)][3]) :
                    down_catch_talk(data , int(i) , output , up = False )
                    
                    output.update({id : (data[str(id)][1] , [output[int(i)][0]] + output[int(i)][1] ) })
                    
            except:
                output.update({id : ( data[str(id)][1] , [] ) })
        if up == True :
            for i in str2list(data[str(id)][2]) :
                up_catch_talk(data , int(i) , output , down = False )
    except :
        return

dataSet = {}
print(str2list("1 ,25,12,40"))
for id in list(cvv.keys())[1:740700] :#[1:743000]
    try :
        id = int(id)
        down_catch_talk(cvv , id ,dataSet)
    except :
        pass
for id in list(cvv.keys())[741400:] :
    try :
        id = int(id)
        down_catch_talk(cvv , id ,dataSet)
    except :
        pass
print(dataSet[1])
# for i in range(1,15):
#     print(cvv[str(i)])
print("Vai salvar")
pickle.dump(dataSet , open("/content/drive/MyDrive/Github_Dir/Chat_Bot/Chat_Bot/datasetDict_Final.pickle","wb"))
print("chegou aqui")

cvv antes :  2800420
cvv depois :  2789265
['1', '25', '12', '40']
(' @115712 I understand. I would like to assist you. We would need to get you into a private secured link to further assist.', [' @sprintcare I have sent several private messages and no one is responding as usual', ' @115712 Please send us a Private Message so that we can further assist you. Just click Message at the top of your profile.', ' @sprintcare I did.', ' @115712 Can you please send us a private message, so that I can gain further details about your account?'])
Vai salvar
chegou aqui


In [ ]:
print(dataSet[2])

In [ ]:

a = {1:4,2:3,4:5}
print(a)
a.update({1:55})
print(a)

{1: 4, 2: 3, 4: 5}
{1: 55, 2: 3, 4: 5}


* *Model :*

In [ ]:
dataSet = pickle.load(open("/content/drive/MyDrive/Github_Dir/Chat_Bot/Chat_Bot/datasetDict_Final.pickle","rb"))
text   = ""
for i in dataSet :
    text += " " + dataSet[i][0]
nGram = nGram_Utils()
print(text[:200])
# nGram.prepare_Text(text, 3 , upper_case = True )
print("Funciona")
print(len(nGram.nGram))

  @115712 Can you please send us a private message, so that I can gain further details about your account?  @sprintcare I did.  @115712 Please send us a Private Message so that we can further assist y
Funciona
86745


In [ ]:
# pickle.dump(nGram , open("/content/drive/MyDrive/Github_Dir/Chat_Bot/Chat_Bot/dataSet_nGram_n=3.pickle","wb"))
print(len(text))
# nGram.prepare_Text(text, 4)
print(len(nGram.nGram2idx))
print(nGram.nGram)
# print(len(nGram.charNgram(text, 4)))
nGram2 = nGram_Utils()

# print(len(nGram2.charNgram(text , 3)))#2: 54009   # 3 : 445693
# decode_string = {}
# var = []
# for i in nGram2.nGram :
#     try :
#         decode_string.update({i.encode('latin-1', "ignore").decode(): None})
#         var += [i.encode('latin-1', "ignore").decode()] 
#     except :
#         pass
# print(nGram2.nGram)
# print(len(decode_string))
# print(var)
# print(var.split())

216642872
86746
('@11', '@1', '@12', '712', '71', '713', '7I3', '7I', '7IO', '7YO', '7Y', '7YH', '7IH', '7AH', '7A', '7AT', '7UT', '7U', '7UI', '7AI', '7AU', '7OU', '7O', '7OR', '7IR', '7UR', '7VR', '7V', '7VM', '7WM', '7W', '7W1', '7WI', '7WL', '7VL', '7BL', '7B', '7BV', '7BU', '7BE', '7BI', '7BO', '7B0', '780', '78', '781', '7B1', '7V1', '7VI', '7VJ', '7UJ', '7EJ', '7E', '7ES', '7YS', '7IS', '7I5', '715', '7L5', '7L', '7L.', '71.', '7I.', '7Y.', '7E.', '7U.', '7V.', '7B.', '78.', '782', '78Z', '7BZ', '7VZ', '7UZ', '7AZ', '7AS', '7A5', '7U5', '7W5', '7WS', '7WX', '7VX', '7BX', '7BK', '78K', '78C', '78Q', '78O', '78U', '78W', '7BW', '7VW', '7UW', '7WW', '7WU', '7UU', '7UA', '7EA', '7FA', '7F', '7FD', '7ED', '7ID', '7UD', '7WD', '7VD', '7BD', '78D', '7TD', '7T', '7TB', '7TV', '7TU', '7TI', '7TO', '7TQ', '7T0', '730', '73', '731', '73L', '73I', '73A', '73E', '73O', '73Q', '73C', '73G', '73J', '732', '73Z', '73S', '735', '733', '723', '72', '722', '72Z', '72S', '7ZS', '7Z', '7ZI', '7ZU', 

In [ ]:
print(len({i.upper() :None for i in nGram2.nGram}))

86745


In [ ]:
n =  pickle.load(open("/content/drive/MyDrive/Github_Dir/Chat_Bot/Chat_Bot/dataSet_nGram_n=3.pickle","rb"))
dataSet = pickle.load(open("/content/drive/MyDrive/Github_Dir/Chat_Bot/Chat_Bot/datasetDict_Final.pickle","rb"))
n.n = 3

print(len(n.nGram))
for i in tuple(dataSet.keys())[:20] :
    print(dataSet[i])
    print(n.text2idx(dataSet[i][0] , n.nGram2idx , n.n ))
    

A saída de streaming foi truncada nas últimas 5000 linhas.
Before_size = -1276338
word : Can
begin : 1
Before_size = -1276339
word : Can
begin : 1
Before_size = -1276340
word : Can
begin : 1
Before_size = -1276341
word : Can
begin : 1
Before_size = -1276342
word : Can
begin : 1
Before_size = -1276343
word : Can
begin : 1
Before_size = -1276344
word : Can
begin : 1
Before_size = -1276345
word : Can
begin : 1
Before_size = -1276346
word : Can
begin : 1
Before_size = -1276347
word : Can
begin : 1
Before_size = -1276348
word : Can
begin : 1
Before_size = -1276349
word : Can
begin : 1
Before_size = -1276350
word : Can
begin : 1
Before_size = -1276351
word : Can
begin : 1
Before_size = -1276352
word : Can
begin : 1
Before_size = -1276353
word : Can
begin : 1
Before_size = -1276354
word : Can
begin : 1
Before_size = -1276355
word : Can
begin : 1
Before_size = -1276356
word : Can
begin : 1
Before_size = -1276357
word : Can
begin : 1
Before_size = -1276358
word : Can
begin : 1
Before_size = -12

In [37]:
print("text"[1:9])

ext


In [2]:
import jaro
import time

class nGram_Utils():
    def __init__(self):
        self.nGram = []
        self.nGram2idx = None
        self.n = None
        self.is_Upper = False
    def prepare_Text(self , text : str , n : int , upper_case = False):
        if upper_case == True :
            text = text.upper()
            self.is_Upper = True
        self.nGram = self.charNgram(text , n )
        self.nGram = self.jaroWinkler_Sort(self.nGram , n )
        
        self.nGram2idx = {self.nGram[i] : i  for i in range(len(self.nGram))}
        self.nGram2idx[" "] = len(self.nGram)

        # return self.text2idx(text , self.nGram2idx , n )
        
    def charNgram(self ,text : str , n : int ) -> list:
        text = text.split()
        # print(" len(text).split == ", len(text))
        output = {}
        for word in text :
            var = { word[i : i + j] : None  for i in range(0 ,len(word) -n + 1 , 1 )  for j in range(1 , n+1)    }
            # print(" len(var) 1 == ", len(var))
            # print(" len(output) 1 == ", len(output))
            
            if len(word)%n != 0 :
                falta = len(word)%n
                var.update({ word[- falta :-i] : None for i in range(1 , falta)})
                var.update({word[- falta :] : None})
            output.update(var)
        # print(" len(output) 2 == ", len(output))
        self.nGram = list(output.keys())
        return self.nGram

    def jaroWinkler_Sort(self ,nGram : list , n :int = None)-> tuple :
        
        if type(n) == type(1) :
            try :
                aux = {i :[] for i in range(1 , n + 1)}
                for i in nGram :
                    aux[len(i)] += [i]
                nGram = []
                for i in list(aux.values() )[::-1] :
                    nGram += i
            except :
                print("Foi identificado que o n passado é menor que o usado na criação do nGram")
        for i in range(len(nGram) - 1 ) :
            var = (jaro.original_metric(nGram[i] , nGram[i + 1] ) , nGram[i+1] ,i + 1 )
            for j in range( i + 1,len(nGram)) :
                if jaro.original_metric(nGram[i] , nGram[j] ) >  var[0] :
                    var = (jaro.original_metric(nGram[i] , nGram[j] ) , nGram[j] , j )
            aux = nGram[i + 1]
            nGram[i + 1] = var[1]
            try :
                nGram[var[2]]= aux
            except :
                print(j)
            
        return tuple(nGram)
    def text2idx(self , text : str , nGram2idx : dict , n )-> tuple :
        '''Se nGram2idx não possui posição do dicionario dedicada ao espaço em 
        branco(" ") este será representado com indíce len(nGram2idx)'''
        text = text.split()
        aux = []
        for word in text :
            begin = 0
            size  = n
            tkn = True
            while tkn :
                try :
                    if begin >= len(word):
                        tkn   = False
                    print(f"word : {word}")
                    print(f"begin : {begin}")
                    print(f"Before_size = {size}")
                    aux   += [nGram2idx[word[begin:begin + size]]]
                    begin += size
                    size   = n
                    print(f"size = {size}")
                    print(f"begin + size : {begin}")
                    print(f"len(word) : {len(word)}")
                    
                    
                        
                except :
                    size -= 1
                if size < 0 :
                    time.sleep(1000)
            try :
              aux += [nGram2idx[" "]]
            except :
              aux += [len(nGram2idx)]
        return tuple(aux[:-1])
un = nGram_Utils()
print(len({1:2,2:3}))
print(un.charNgram("cincocinsa" , 4 ))
print(un.jaroWinkler_Sort(un.charNgram("cincocinsa" , 4 ) , 4 ))
print(jaro.jaro_metric("abco","bc"))

2
['c', 'ci', 'cin', 'cinc', 'i', 'in', 'inc', 'inco', 'n', 'nc', 'nco', 'ncoc', 'co', 'coc', 'coci', 'o', 'oc', 'oci', 'ocin', 'cins', 'ins', 'insa', 's', 'sa']
('cinc', 'cin', 'cins', 'ins', 'insa', 'in', 'inc', 'inco', 'nco', 'ncoc', 'coc', 'coci', 'co', 'c', 'ci', 'ocin', 'oci', 'oc', 'o', 'nc', 'n', 'i', 'sa', 's')
0.8333333333333334


* *Github Save :*

In [ ]:
# path = "/content/drive/MyDrive/Colab Notebooks/Chat_Bot.ipynb"
# !rsync -aP "{path}" /content/Chat_Bot

# !git clone "{Git_CB_Path}" ./temp
# !rsync -aP /content/Chat_Bot/* ./temp
# %cd ./temp
# !git add -u
# !git commit -m "update"
# !git config user.email "limaalyson@hotmail.com"
# !git config user.name "Alyson_Google_Colab"
# !git push origin master
# %cd /content
# !rm -rf ./temp
